# README

After running the 1st block, you need to pick:
    - a dataset, 
    - a type of feature
    - a model
    - a loss function
then run the "main loop" block 

In [ ]:
# run this first
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time

x_min, x_max = -2, 4
y_min, y_max = -5, 3

x_mesh, y_mesh = np.meshgrid(np.arange(x_min, x_max, .05), np.arange(y_min, y_max, .05))
X_mesh0 = np.vstack((x_mesh.ravel(), y_mesh.ravel())).T

# Plot also the training points
def plot_data(X, Y):
    plt.axis('off')
    plt.scatter(X[:, 0], X[:, 1], c=Y, cmap='winter', s =.5, marker='x')
    plt.axis([x_min, x_max, y_min, y_max])

def plot_decision_boundary(Z):
    plt.contourf(x_mesh, y_mesh, Z > 0, cmap=plt.cm.Paired)

# Dataset

each cell below loads a different dataset.

In [ ]:
fn = 'linearly_separable.npz'
data = np.load('./'+fn)
X0 = data['X']
Y = data['Y']
plt.figure(dpi=100)
plt.title(fn)
plot_data(X0,Y)
plt.show()

In [ ]:
fn = 'double_moon.npz'
data = np.load('./'+fn)
X0 = data['X']
Y = data['Y']
plt.figure(dpi=100)
plt.title(fn)
plot_data(X0,Y)
plt.show() 

# Features

Each cell below contains a different processing of the features: linear or polynomials

In [ ]:
## linear features
feature_name = 'Linear features'
X = X0
X_mesh = X_mesh0

In [ ]:
## polynomial features
feature_name = 'Polynomial features'

def polynomial_transform(X, d):
    nSamples = X.shape[0]
    # transform (X_1,X_2) -> (X_1,X_2, X_1*X_2, X_1^2,..,X_2^d)
    XX = X
    for j in range(d):
        for k in range(d):
            XX = np.hstack((np.reshape((X[:, 0]**j)*(X[:, 1]**k), (nSamples, 1)), XX)) 
    return XX

d = 5 #degree of the polynom
X = polynomial_transform(X0, d)
X_mesh = polynomial_transform(X_mesh0, d)

# Models

Each cell below contains a different model: Perceptron, logistic regression.


In [ ]:
## Linear model 
class Model:
    def __init__(self, x):
        self.name = 'Linear'
        self.w = np.zeros(x.shape[1])
        self.b = 0.
    # compute prediction
    def forward (self, x):
        return np.dot(x, self.w) + self.b 
    # compute gradient
    def backward (self, x, err):
        self.dw = np.dot(err, x) / x.shape[0]
        self.db = np.mean(err)
    #update parameters
    def update(self, eps):
        self.w += eps * self.dw
        self.b += eps * self.db

In [ ]:
## 2 layer neural network
h = 100
class Model:
    def __init__(self, x):
        self.name='2 layer NN'
        self.w_i = np.random.randn(x.shape[1], h)
        self.w_o = np.zeros(h)
        self.b_i = np.zeros(h)
        self.b_o = 0.
        
    def forward(self, x):
        self.hidden = np.maximum(np.dot(x, self.w_i) + self.b_i , 0)
        return np.dot(self.hidden, self.w_o) + self.b_o
    
    def backward(self, x, err):
        self.dw_o = np.dot(err, self.hidden) / x.shape[0]
        self.db_o = np.mean(err)
        err_hidden = np.multiply(np.outer(err,self.w_o), (self.hidden)>0)
        self.dw_i = np.dot(err_hidden.T,x).T / x.shape[0]
        self.db_i = np.mean(err_hidden)
        
    def update(self, eps):
        self.w_i += eps * self.dw_i
        self.b_i += eps * self.db_i
        self.w_o += eps * self.dw_o
        self.b_o += eps * self.db_o
        

# Loss function

Each cell below contains a different loss function.

In [ ]:
## Perceptron 
class Loss:
    def __init__(self):
        self.name = 'Perceptron'
    # compute prediction
    def forward (self, x):
        return x > 0 
    # compute gradient
    def backward (self, pred, y):
        return y - pred

In [ ]:
## logistic regression 
class Loss:
    def __init__(self):
        self.name = 'Logistic Regression'
    # compute prediction
    def forward (self, x):
        idx = np.where(x < -20)
        pred = 1. / (1 + np.exp(-x))
        pred[idx] = 0
        return pred
    # compute gradient
    def backward (self,pred, y):
        return y - pred

# Main loop


In [ ]:
## main loop for training
plt.figure(dpi=150)

model = Model(X)
loss = Loss()

eps = 1
for i in range(1,1001): 
    
    out  = model.forward (X)
    pred = loss.forward(out)  
    
    err = loss.backward(pred, Y)
    model.backward(X, err)
    
    model.update(eps)
    
    err = np.mean(Y != (pred>0.5))
    
    # Draw figure:
    Z = model.forward(X_mesh)
    Z = Z.reshape(x_mesh.shape)
    title = model.name+'model with '+loss.name+' on '+feature_name+': Iteration = ' + str(i) + ', Error = ' + str(err)   

    plt.clf() 
    plt.title(title)
    plot_decision_boundary(Z)
    plot_data(X0,Y)
    display.clear_output(wait=True)
    display.display(plt.gcf())
    
    if err <= 0.001:
        break
        
display.clear_output(wait=True)